In [3]:
from datetime import datetime
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())
# spark.sparkContext.getConf().getAll()

In [4]:
products = spark.read.format("parquet").load("s3a://warehouse/gold/tiki/products.parquet")
users = spark.read.format("parquet").load("s3a://warehouse/gold/tiki/users.parquet")
reviews = spark.read.format("parquet").load("s3a://warehouse/gold/tiki/reviews.parquet")

In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from underthesea import word_tokenize
from pyspark.sql.functions import concat, lit

import re

@udf(StringType()) 
def process_text(document):
    # Change to lowercase
    document = document.lower()
    
    # Remove HTTP links (using regular expression)
    document = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', document)
    
    # Remove line breaks (replace with space)
    document = re.sub(r'[\r\n]+', ' ', document)
    
    # Replace '/' and ',' with space
    document = document.replace('/', ' ').replace(',', ' ')
    
    # Remove punctuations using regular expression
    document = re.sub(r'[^\w\s]', '', document)
    
    # Remove extra spaces (replace multiple spaces with a single space)
    document = re.sub(r'[\s]{2,}', ' ', document)
    
    # Tokenize text using word_tokenize from underthesea
    document = word_tokenize(document, format="text")
    
    return document



# Create the 'info' column by concatenating product_name, description, and specifications
products = products.withColumn(
    'info',
    concat(products['product_name'], lit(' '), products['description'], lit(' '), products['specifications'])
)
# Apply the UDF to the 'info' column
products = products.withColumn('processed_info', process_text(products['info']))

# # Show the results
# products.select('processed_info').show()

In [6]:
from gensim import corpora, models, similarities

def load_stopword(STOP_WORDS):
    with open(STOP_WORDS, 'r', encoding = 'utf-8') as file:
        stop_words = file.read()
    stop_words = stop_words.split('\n')
    return stop_words
    
def gensim_rcm(gold_products, stop_words):
    
    gold_products = gold_products.withColumn(
        'content',
        concat(gold_products['product_name'], lit(' '), gold_products['description'], lit(' '), gold_products['specifications'])
    )

    gold_products = gold_products.withColumn('processed_info', process_text(gold_products['content']))

    info = gold_products.select("processed_info").rdd.map(lambda row: row[0].split()).collect()

    dictionary = corpora.Dictionary(info)

    stop_words = load_stopword("vietnamese-stopwords.txt")
    stop_ids = [dictionary.token2id[stopword] for stopword in stop_words if stopword in dictionary.token2id]
    once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
    dictionary.filter_tokens(stop_ids + once_ids)
    dictionary.compactify()

    corpus = [dictionary.doc2bow(text) for text in info]

    tfidf = models.TfidfModel(corpus)

    feature_cnt = len(dictionary.token2id)
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=feature_cnt)
    
    # Return the dictionary, TF-IDF model, and similarity index for further use
    dictionary.save('dictionary.gensim')
    tfidf.save('tfidf_model.gensim')
    index.save('similarity_index.gensim')
    return dictionary, tfidf, index

stop_words = load_stopword("vietnamese-stopwords.txt")
# _ = gensim_rcm(products, stop_words)
tfidf = models.TfidfModel.load('tfidf_model.gensim')
index = similarities.SparseMatrixSimilarity.load('similarity_index.gensim')
dictionary = corpora.Dictionary.load('dictionary.gensim')

In [8]:
# dictionary.save("s3a://warehouse/gold/tiki/dict.gensim")

In [11]:
import pandas as pd
def get_id_rcm(n, product_id, seller_id, dictionary, tfidf,index, df):

    product_result = df[(df['product_id'] == product_id) & (df['seller_id'] == seller_id)]
    view_product = product_result['processed_info'].values[0]
    view_product = view_product.split()

    bow_vector = dictionary.doc2bow(view_product)
    sim = index[tfidf[bow_vector]]
    
    df_result = pd.DataFrame({'id': range(len(sim)), 'score': sim})
    
    top_scores = df_result.sort_values(by='score', ascending=False).head(n + 1)
    
    product_find = df[df.index.isin(top_scores['id'])]
    
    result = pd.concat([product_find[['product_id', 'seller_id', 'product_name']], top_scores], axis=1)
    
    result = result[(result['product_id'] != product_id) | (result['seller_id'] != seller_id)]
    
    recommended_ids = result.sort_values(by='score', ascending=False)['id'].tolist()
    return result

get_id_rcm(10,11708848,14626,dictionary,tfidf, index, products.toPandas())

,product_id,seller_id,product_name,id,score
90,706939,53660,The Subtle Art of Not Giving a F*Ck : A Counte...,90,0.271541
415,11778902,14626,The 5 AM Club: Own Your Morning. Elevate Your ...,415,0.247070
521,17964102,14626,The Classical Music Book,521,0.239234
2931,170610748,14626,Sách - Eat That Frog! 21 Great Ways to Stop Pr...,2931,0.241484
2957,170726346,14626,Sách - Greenlights by Matthew McConaughey (US ...,2957,0.247357
5947,11779322,14626,Where You Go Is Not Who You'll Be: An Antidote...,5947,0.252166
5953,11842316,14626,Gravity Falls Dipper's And Mabel's Guide To My...,5953,0.257969
8479,170678920,14626,Sách - The Gifts of Imperfection: 10th Anniver...,8479,0.236065
9928,271394768,14626,Sách - A Swim in a Pond in the Rain : From the...,9928,0.252176
10176,273752630,14626,Sách - The World Atlas of Coffee : From beans ...,10176,0.258908


## ALS

In [38]:
from pyspark.sql.functions import explode, col, split, concat, lit, expr
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer

def rcm_collaborative_filtering(gold_reviews, n = 10):

    gold_reviews = gold_reviews.withColumn(
        'item_id',
        concat(gold_reviews['product_id'], lit('_'), gold_reviews['seller_id'])
    )

    gold_reviews = gold_reviews.withColumn('original_item_id', gold_reviews['item_id'])

    indexer = StringIndexer(inputCol="item_id", outputCol="item_index")
    gold_reviews = indexer.fit(gold_reviews).transform(gold_reviews)

    als = ALS(userCol="customer_id", itemCol="item_index", ratingCol="rating", coldStartStrategy="drop")
    model = als.fit(gold_reviews)

    user_recs = model.recommendForAllUsers(n)

    exploded_recs = user_recs.withColumn("recommendation", explode(col("recommendations")))
    exploded_recs = exploded_recs.withColumn("item_index", col("recommendation.item_index"))
    exploded_recs = exploded_recs.withColumn("rating", col("recommendation.rating"))
    
    distinct_items = gold_reviews.select("original_item_id", "item_index").dropDuplicates(["item_index"])
    processed_user_recs = exploded_recs.join(
        distinct_items,
        on="item_index"
    )

    processed_user_recs = processed_user_recs.withColumn("product_id", split(col("original_item_id"), "_")[0].cast("int"))
    processed_user_recs = processed_user_recs.withColumn("seller_id", split(col("original_item_id"), "_")[1].cast("int"))


    processed_user_recs = processed_user_recs.select(
        "customer_id",
        "product_id",
        "seller_id",
        "rating"
    )

    processed_user_recs.show()
    
    return processed_user_recs

obj = rcm_collaborative_filtering(reviews, n = 20)


+-----------+----------+---------+---------+
|customer_id|product_id|seller_id|   rating|
+-----------+----------+---------+---------+
|         27|    650250|        1| 4.941955|
|         27| 118130919|   117842| 4.604446|
|         27| 186364165|    96252|  4.22681|
|         27| 146773059|    47984| 4.189372|
|         27| 117927305|     2322|4.1005893|
|         27| 211946959|    90053|4.0803447|
|         27| 172283470|    18138| 4.012916|
|         27| 272030678|   313025|3.9958537|
|         27| 275302988|   261473|3.9737282|
|         27|  13356120|    20142|3.9696078|
|         27|  54439144|        1|  3.96342|
|         27| 103209852|   191019|3.9271016|
|         27| 174619837|     3778|3.9253488|
|         27| 165700192|        1|3.9191248|
|         27|  68785171|     9612|3.8994894|
|         27| 192927412|   206733|3.8965356|
|         27| 161459742|   249806|3.8853023|
|         27| 163576573|     1130|3.8748899|
|         27|  67827661|     4106|  3.82903|
|         

In [39]:
obj.write.mode("overwrite").parquet("s3a://warehouse/recommendation/tiki/als.parquet")

In [40]:
import pandas as pd
als_data = spark.read.format("parquet").load("s3a://warehouse/recommendation/tiki/als.parquet")
als_data = als_data.toPandas()

In [41]:
als_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2678180 entries, 0 to 2678179
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   customer_id  int32  
 1   product_id   int32  
 2   seller_id    int32  
 3   rating       float32
dtypes: float32(1), int32(3)
memory usage: 40.9 MB


In [42]:
def als_recommendation(customer_id, data_als, data):
    als_result = data_als[data_als["customer_id"] == customer_id]
    result = pd.merge(als_result, data, on = ["product_id","seller_id"], how = "left")
    return result[["customer_id", "product_id","seller_id", "product_name"]]
    # return list(result["product_id"])

In [44]:
data = products.toPandas()

In [22]:
data = data[["product_id","seller_id"]].astype(int)
als_result = als_data[als_data.customer_id == 8097347]


In [25]:
als_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2812089 entries, 0 to 2812088
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   customer_id  int32  
 1   product_id   object 
 2   seller_id    object 
 3   rating       float32
dtypes: float32(1), int32(1), object(2)
memory usage: 64.4+ MB


In [45]:
als_recommendation(8097347,als_data,data)

,customer_id,product_id,seller_id,product_name
0,8097347,1695753,138278,Máy Sấy Tóc Philips HP8108/00 - Hàng Chính Hãng
1,8097347,273402112,1,Sữa rửa mặt giúp làm sạch sâu dành cho da dầu ...
2,8097347,213912311,306444,Chân đế máy giặt Electrolux nâng máy lên cao t...
3,8097347,32505959,100773,Remote điều khiển đa năng - Dùng được cho tất ...
4,8097347,1687625,1,Kem chống muỗi Rohto Metholatum Remos Hương Sả...
5,8097347,2752195,103815,Đồng hồ Nam thể thao SKMEI 1155B - DHA473
6,8097347,155925409,165363,Đồng Hồ Nam Dây Nhựa Casio Standard AE-1500WH-...
7,8097347,106390917,195148,Tặng Kèm Tovit Thay Dây đồng hồ Hublot cao su ...
8,8097347,273250656,1,CHÌ KẺ CHÂN MÀY INNISFREE AUTO EYEBROW PENCIL ...
9,8097347,212213013,1,Từ điển tiếng việt dành cho học sinh - khổ to ...


In [2]:
!pip install minio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.6 MB/s eta 0:00:00a 0:00:01


In [3]:
import pandas as pd
from contextlib import contextmanager
from sqlalchemy import create_engine, text
from minio import Minio
from gensim import corpora, models, similarities
from underthesea import word_tokenize
import os 

MINIO_CONFIG = {
    "endpoint_url": "minio:9000",
    "bucket": "warehouse",
    "aws_access_key_id": "minio",
    "aws_secret_access_key": "minio123",
}

PSQL_CONFIG = {
    "host": "de_psql",
    "port": 5432,
    "database": "postgres",
    "user": "admin",
    "password": "admin123",
}

@contextmanager
def connect_psql(config):
    conn_info = (
        f"postgresql+psycopg2://{config['user']}:{config['password']}"
        + f"@{config['host']}:{config['port']}"
        + f"/{config['database']}"
    )
    db_conn = create_engine(conn_info)
    try:
        yield db_conn
    except Exception:
        raise

@contextmanager
def connect_minio(config):
    client = Minio(
        endpoint=config.get("endpoint_url"),
        access_key=config.get("aws_access_key_id"),
        secret_key=config.get("aws_secret_access_key"),
        secure=False,
    )
    try:
        yield client
    except Exception:
        raise

In [22]:
from datetime import datetime
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

products = spark.read.format("parquet").load("s3a://warehouse/gold/tiki/products.parquet")

In [33]:
# products_spark = products.toPandas()
# products_spark= products_spark.sort_values(by=['product_id','seller_id'])
products = products.orderBy(["product_id", "seller_id"])
products.toPandas()

,category_id,product_id,seller_id,brand_id,product_name,description,specifications,breadcrumbs,original_price,discount,...,day_ago_created,product_url,is_authentic,is_freeship_xtra,is_top_deal,return_reason,inventory_type,warranty_period,warranty_type,warranty_location
0,1815,54665,1,18984.0,Chuột không dây Logitech M185 - Hãng chính hãng,Tận hưởng thế giới không dây\nChuột không dây ...,"Hệ điều hành: Windows Vista®, Windows® 7, Wind...",Thiết Bị Số - Phụ Kiện Số / Phụ kiện máy tính ...,289000,70000,...,4364,https://tiki.vn/product-p54665.html?spid=54673,True,False,False,Bất cứ lý do gì,backorder,1080,Không bảo hành,Không bảo hành
1,1815,122012,1,18984.0,Combo chuột phím không dây Logitech MK240 - Hà...,Kết nối không dây tin cậy\nCombo chuột phím...,Thương hiệu: Logitech\nXuất xứ thương hiệu: Th...,Thiết Bị Số - Phụ Kiện Số / Phụ kiện máy tính ...,620000,161000,...,3671,https://tiki.vn/product-p122012.html?spid=122015,True,False,True,Bất cứ lý do gì,backorder,1080,Điện tử,Bảo hành chính hãng
2,1815,125781,1,18641.0,Tai Nghe Nhét Tai Sony Fontopia MDR-E9LP - Hàn...,\nThiết kế gọn nhẹ\nTai Nghe Sony Fontopia MDR...,Thương hiệu: Sony\nXuất xứ thương hiệu: Nhật B...,Thiết Bị Số - Phụ Kiện Số / Thiết Bị Âm Thanh ...,159000,20000,...,3628,https://tiki.vn/product-p125781.html?spid=125798,True,False,True,Bất cứ lý do gì,instock,360,Phiếu bảo hành,Bảo hành chính hãng
3,1815,126003,1,18641.0,Tai Nghe Chụp Tai Sony MDR-ZX310AP - Hàng Chín...,"Thiết kế phần housing rất bóng bẩy, hạn chế tr...",Thương hiệu: Sony\nXuất xứ thương hiệu: Nhật B...,Thiết Bị Số - Phụ Kiện Số / Thiết Bị Âm Thanh ...,890000,200000,...,3627,https://tiki.vn/product-p126003.html?spid=126129,True,False,False,Bất cứ lý do gì,backorder,360,Điện tử,Bảo hành chính hãng
4,1815,126697,1,18641.0,Tai Nghe Nhét Tai Sony MDR-EX15AP - Hàng Chính...,Dể dàng điều khiển\nTai Nghe Sony MDREX15AP - ...,Thương hiệu: Sony\nXuất xứ thương hiệu: Nhật B...,Thiết Bị Số - Phụ Kiện Số / Thiết Bị Âm Thanh ...,199000,0,...,3620,https://tiki.vn/product-p126697.html?spid=126709,True,False,True,Bất cứ lý do gì,instock,360,Phiếu bảo hành,Bảo hành chính hãng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11120,1815,276782048,313025,21653.0,Củ Sạc Nhanh Cho Iphone Hoco C80/C80Plus/CS13/...,Củ sạc nhanh 20w C80 chính hãng cho iphone PD ...,Thương hiệu: Hoco\nXuất xứ thương hiệu: Hong K...,Thiết Bị Số - Phụ Kiện Số / Phụ Kiện Điện Thoạ...,129000,0,...,7,https://tiki.vn/product-p276782048.html?spid=2...,True,True,False,Bất cứ lý do gì,instock,90,Tem bảo hành,Bảo hành bởi nhà bán hàng thông qua Tiki
11121,1789,276799143,10016,18802.0,Máy Tính Bảng Samsung Galaxy Tab S9 FE Wi-Fi (...,\n*Hình ảnh minh họa. Màu sắc và kiểu máy Gala...,Dung lượng pin: 8000 mAh\nBluetooth: v5.3\nThư...,Điện Thoại - Máy Tính Bảng / Máy tính bảng,9990000,2000000,...,6,https://tiki.vn/product-p276799143.html?spid=2...,True,True,False,Bất cứ lý do gì,backorder,360,Điện tử,Bảo hành chính hãng
11122,1789,276802409,162514,18802.0,Điện thoại Samsung Galaxy A16 5G - Hàng chính ...,"\nMàn hình: Super AMOLED 6.7 inch, FHD+, 90Hz\...",Thương hiệu: Samsung\nXuất xứ (Made in): Hàn Q...,Điện Thoại - Máy Tính Bảng / Điện thoại Smartp...,6090000,291000,...,5,https://tiki.vn/product-p276802409.html?spid=2...,True,True,False,Bất cứ lý do gì,backorder,0,Không bảo hành,Không bảo hành
11123,1815,276829576,313025,21653.0,CỐC SẠC NHANH 12W 2 CỔNG CHỐNG CHÁY NỔ HOCO C7...,\nCủ Sạc Nhanh 2 Cổng USB C73 Dòng ra 2.4A\nĐ...,Sản phẩm có được bảo hành không?: Có\nCó thuế ...,Thiết Bị Số - Phụ Kiện Số / Phụ Kiện Điện Thoạ...,69000,0,...,1,https://tiki.vn/product-p276829576.html?spid=2...,True,True,False,Bất cứ lý do gì,instock,90,Tem bảo hành,Bảo hành bởi nhà bán hàng thông qua Tiki


In [35]:
with connect_psql(PSQL_CONFIG) as db_conn:
    sql_stm = "SELECT * FROM dwh.products"
    products_df = pd.read_sql(sql_stm,db_conn)
products_df = products_df.astype({'product_id': 'int', 'seller_id': 'int'})  
products_df = products_df.sort_values(by=['product_id', 'seller_id']).reset_index()
products_df

,index,product_id,seller_id,category_id,brand_id,product_name,description,specifications,breadcrumbs,original_price,...,day_ago_created,product_url,is_authentic,is_freeship_xtra,is_top_deal,return_reason,inventory_type,warranty_period,warranty_type,warranty_location
0,0,54665,1,1815,18984,Chuột không dây Logitech M185 - Hãng chính hãng,Tận hưởng thế giới không dây\nChuột không dây ...,"Hệ điều hành: Windows Vista®, Windows® 7, Wind...",Thiết Bị Số - Phụ Kiện Số / Phụ kiện máy tính ...,289000.0,...,4364,https://tiki.vn/product-p54665.html?spid=54673,True,False,False,Bất cứ lý do gì,backorder,1080,Không bảo hành,Không bảo hành
1,1255,122012,1,1815,18984,Combo chuột phím không dây Logitech MK240 - Hà...,Kết nối không dây tin cậy\nCombo chuột phím...,Thương hiệu: Logitech\nXuất xứ thương hiệu: Th...,Thiết Bị Số - Phụ Kiện Số / Phụ kiện máy tính ...,620000.0,...,3671,https://tiki.vn/product-p122012.html?spid=122015,True,False,True,Bất cứ lý do gì,backorder,1080,Điện tử,Bảo hành chính hãng
2,1232,125781,1,1815,18641,Tai Nghe Nhét Tai Sony Fontopia MDR-E9LP - Hàn...,\nThiết kế gọn nhẹ\nTai Nghe Sony Fontopia MDR...,Thương hiệu: Sony\nXuất xứ thương hiệu: Nhật B...,Thiết Bị Số - Phụ Kiện Số / Thiết Bị Âm Thanh ...,159000.0,...,3628,https://tiki.vn/product-p125781.html?spid=125798,True,False,True,Bất cứ lý do gì,instock,360,Phiếu bảo hành,Bảo hành chính hãng
3,1214,126003,1,1815,18641,Tai Nghe Chụp Tai Sony MDR-ZX310AP - Hàng Chín...,"Thiết kế phần housing rất bóng bẩy, hạn chế tr...",Thương hiệu: Sony\nXuất xứ thương hiệu: Nhật B...,Thiết Bị Số - Phụ Kiện Số / Thiết Bị Âm Thanh ...,890000.0,...,3627,https://tiki.vn/product-p126003.html?spid=126129,True,False,False,Bất cứ lý do gì,backorder,360,Điện tử,Bảo hành chính hãng
4,1,126697,1,1815,18641,Tai Nghe Nhét Tai Sony MDR-EX15AP - Hàng Chính...,Dể dàng điều khiển\nTai Nghe Sony MDREX15AP - ...,Thương hiệu: Sony\nXuất xứ thương hiệu: Nhật B...,Thiết Bị Số - Phụ Kiện Số / Thiết Bị Âm Thanh ...,199000.0,...,3620,https://tiki.vn/product-p126697.html?spid=126709,True,False,True,Bất cứ lý do gì,instock,360,Phiếu bảo hành,Bảo hành chính hãng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11120,11123,276782048,313025,1815,21653,Củ Sạc Nhanh Cho Iphone Hoco C80/C80Plus/CS13/...,Củ sạc nhanh 20w C80 chính hãng cho iphone PD ...,Thương hiệu: Hoco\nXuất xứ thương hiệu: Hong K...,Thiết Bị Số - Phụ Kiện Số / Phụ Kiện Điện Thoạ...,129000.0,...,7,https://tiki.vn/product-p276782048.html?spid=2...,True,True,False,Bất cứ lý do gì,instock,90,Tem bảo hành,Bảo hành bởi nhà bán hàng thông qua Tiki
11121,9634,276799143,10016,1789,18802,Máy Tính Bảng Samsung Galaxy Tab S9 FE Wi-Fi (...,\n*Hình ảnh minh họa. Màu sắc và kiểu máy Gala...,Dung lượng pin: 8000 mAh\nBluetooth: v5.3\nThư...,Điện Thoại - Máy Tính Bảng / Máy tính bảng,9990000.0,...,6,https://tiki.vn/product-p276799143.html?spid=2...,True,True,False,Bất cứ lý do gì,backorder,360,Điện tử,Bảo hành chính hãng
11122,9642,276802409,162514,1789,18802,Điện thoại Samsung Galaxy A16 5G - Hàng chính ...,"\nMàn hình: Super AMOLED 6.7 inch, FHD+, 90Hz\...",Thương hiệu: Samsung\nXuất xứ (Made in): Hàn Q...,Điện Thoại - Máy Tính Bảng / Điện thoại Smartp...,6090000.0,...,5,https://tiki.vn/product-p276802409.html?spid=2...,True,True,False,Bất cứ lý do gì,backorder,0,Không bảo hành,Không bảo hành
11123,11124,276829576,313025,1815,21653,CỐC SẠC NHANH 12W 2 CỔNG CHỐNG CHÁY NỔ HOCO C7...,\nCủ Sạc Nhanh 2 Cổng USB C73 Dòng ra 2.4A\nĐ...,Sản phẩm có được bảo hành không?: Có\nCó thuế ...,Thiết Bị Số - Phụ Kiện Số / Phụ Kiện Điện Thoạ...,69000.0,...,1,https://tiki.vn/product-p276829576.html?spid=2...,True,True,False,Bất cứ lý do gì,instock,90,Tem bảo hành,Bảo hành bởi nhà bán hàng thông qua Tiki
